In [ ]:
import os
os.kill(os.getpid(), 9)


In [1]:
import os

# Mostrar o diretório de trabalho atual
print(os.getcwd())
# print(os.listdir('./cc09-cow-project/'))


/mnt/sda3/home2/tony-sousa/cc09-cow-project


In [3]:
# Instalação das bibliotecas necessárias
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install tqdm
!pip install h5py
!pip install xmltodict
!pip install xml.etree.ElementTree

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-2.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)
Using cached numpy-2.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
  Using cached matplotlib-3.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (163 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-10.4.0-cp312-cp312-manylinu

In [3]:
# Listar arquivos e diretórios

# folder_path = './cc09-cow-project'
folder_path = '.'

In [1]:
import cv2
import numpy as np
import os
import h5py
import xml.etree.ElementTree as ET  # Considere usar 'from lxml import etree as ET' para melhor desempenho
from tqdm import tqdm

# ======= 1. Configurações Iniciais =======

# Caminhos para os vídeos e o arquivo de anotações XML
folder_path = '.' 
color_video_path = os.path.join(folder_path, 'videos', 'video_colorido_concatenado_10fps.mp4')
thermal_video_path = os.path.join(folder_path, 'videos', 'video_preto_branco_10fps.mp4')
annotations_path = os.path.join(folder_path, 'xml', 'annotations_colorido_correct.xml')

# Diretório para salvar os arquivos .h5
data_dir = os.path.join(folder_path, 'dados')
os.makedirs(data_dir, exist_ok=True)
masked_color_frames_h5_path = os.path.join(data_dir, 'masked_color_frames.h5')
color_frames_h5_path = os.path.join(data_dir, 'color_frames.h5')
thermal_frames_h5_path = os.path.join(data_dir, 'thermal_frames.h5')

# ======= 2. Funções Auxiliares =======

def extract_frames(video_path, start_frame, end_frame):
    """
    Extrai frames de um vídeo em um intervalo específico.

    Args:
        video_path (str): Caminho para o arquivo de vídeo.
        start_frame (int): Frame inicial.
        end_frame (int): Frame final.

    Returns:
        tuple: (frames, frame_names)
            - frames (list): Lista de frames extraídos.
            - frame_names (list): Lista de nomes dos frames.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erro: Não foi possível abrir o vídeo: {video_path}")
        return [], []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frames = []
    frame_names = []

    for idx in tqdm(range(start_frame, end_frame), desc=f"Extraindo frames de {os.path.basename(video_path)} - Parte {start_frame} a {end_frame}"):
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Frame {idx} não pôde ser lido. Parando a extração.")
            break
        frames.append(frame)
        # Extrair o nome do frame a partir do índice
        frame_name = f"frame_{idx:06d}"
        frame_names.append(frame_name)

    cap.release()
    return frames, frame_names

def parse_annotations(annotations_path, relevant_frames, image_shape):
    """
    Parseia o arquivo XML de anotações e cria máscaras para os frames relevantes.

    Args:
        annotations_path (str): Caminho para o arquivo XML de anotações.
        relevant_frames (set): Conjunto de IDs de frames relevantes.
        image_shape (tuple): Forma da imagem (altura, largura, canais).

    Returns:
        dict: Dicionário mapeando números de frames para suas máscaras combinadas.
    """
    tree = ET.parse(annotations_path)
    root = tree.getroot()

    masks_per_frame = {}

    for image in tqdm(root.findall('image'), desc="Parseando anotações XML"):
        frame_id = int(image.attrib['id'])
        if frame_id not in relevant_frames:
            continue  # Ignorar frames não relevantes
        for polygon in image.findall('polygon'):
            label = polygon.attrib.get('label', 'unknown')
            points_str = polygon.attrib['points']
            points = []
            for point in points_str.split(';'):
                x_str, y_str = point.strip().split(',')
                x = float(x_str)
                y = float(y_str)
                points.append((x, y))
            # Criar máscara para o polígono atual
            single_mask = np.zeros(image_shape[:2], dtype=np.uint8)
            cv2.fillPoly(single_mask, [np.array(points, dtype=np.int32)], color=255)

            if frame_id in masks_per_frame:
                # Combinar a nova máscara com a existente usando operação lógica OR
                masks_per_frame[frame_id] = cv2.bitwise_or(masks_per_frame[frame_id], single_mask)
            else:
                masks_per_frame[frame_id] = single_mask

    return masks_per_frame

def apply_masks(color_frames, masks_per_frame, frame_ids):
    """
    Aplica as máscaras aos frames coloridos.

    Args:
        color_frames (list): Lista de frames coloridos.
        masks_per_frame (dict): Dicionário mapeando números de frames para suas máscaras.
        frame_ids (list): Lista de IDs dos frames correspondentes aos color_frames.

    Returns:
        tuple: (masked_color_frames, frames_with_masks_indices)
            - masked_color_frames (list): Lista de frames coloridos com máscaras aplicadas.
            - frames_with_masks_indices (list): Lista de números dos frames que possuem máscaras.
    """
    masked_color_frames = []
    frames_with_masks_indices = []

    for i in tqdm(range(len(color_frames)), desc="Aplicando máscaras aos frames coloridos"):
        frame_id = frame_ids[i]
        if frame_id in masks_per_frame:
            mask = masks_per_frame[frame_id]
            frame = color_frames[i]
            # Verificar se as dimensões da máscara correspondem às do frame
            if mask.shape != frame.shape[:2]:
                print(f"Warning: A máscara para o frame {frame_id} tem dimensões {mask.shape}, mas o frame colorido tem dimensões {frame.shape[:2]}. Ignorando máscara para este frame.")
                continue
            # Aplicar máscara ao frame (sobrepor em vermelho)
            masked_frame = frame.copy()
            masked_frame[mask == 255] = [0, 0, 255]  # Cor vermelha para a máscara
            masked_color_frames.append(masked_frame)
            frames_with_masks_indices.append(frame_id)
        else:
            # Nenhuma máscara para este frame
            pass

    return masked_color_frames, frames_with_masks_indices

def save_frames_to_h5(h5f, dataset_name, frames):
    """
    Salva frames em um arquivo .h5 em um dataset específico.

    Args:
        h5f (h5py.File): Arquivo HDF5 aberto.
        dataset_name (str): Nome do dataset dentro do arquivo .h5.
        frames (list ou np.ndarray): Lista ou array de frames a serem salvos.
    """
    frames_np = np.array(frames)
    if dataset_name in h5f:
        print(f"Atualizando o dataset existente: {dataset_name}")
        del h5f[dataset_name]  # Remove se já existir
    h5f.create_dataset(dataset_name, data=frames_np, compression="gzip", compression_opts=4)  # Ajuste o nível de compressão conforme necessário
    print(f"Frames salvos no dataset: {dataset_name}")

def save_frame_names(h5f, dataset_name, frame_names):
    """
    Salva os nomes dos frames em um dataset específico dentro de um arquivo .h5.

    Args:
        h5f (h5py.File): Arquivo HDF5 aberto.
        dataset_name (str): Nome do dataset dentro do arquivo .h5.
        frame_names (list): Lista de nomes dos frames.
    """
    # Convertendo para bytes para armazenar strings no HDF5
    frame_names_bytes = [name.encode('utf-8') for name in frame_names]
    dt = h5py.string_dtype(encoding='utf-8')
    if dataset_name in h5f:
        print(f"Atualizando o dataset existente: {dataset_name}")
        del h5f[dataset_name]
    h5f.create_dataset(dataset_name, data=frame_names_bytes, dtype=dt)
    print(f"Nomes dos frames salvos no dataset: {dataset_name}")

# ======= 3. Execução Principal =======

# 3.1. Abrir os vídeos e obter o número total de frames
color_cap = cv2.VideoCapture(color_video_path)
thermal_cap = cv2.VideoCapture(thermal_video_path)
total_color_frames = int(color_cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_thermal_frames = int(thermal_cap.get(cv2.CAP_PROP_FRAME_COUNT))
color_cap.release()
thermal_cap.release()

# 3.2. Verificar se os vídeos têm o mesmo número de frames
if total_color_frames != total_thermal_frames:
    print(f"Atenção: O vídeo colorido tem {total_color_frames} frames e o vídeo térmico tem {total_thermal_frames} frames.")
    total_frames = min(total_color_frames, total_thermal_frames)
else:
    total_frames = total_color_frames

# 3.3. Dividir o vídeo em menos partes para reduzir overhead
num_chunks = 3 
chunk_size = total_frames // num_chunks

# 3.4. Processamento dos Frames Coloridos e Aplicação de Máscaras

print("\n=== Processamento dos Frames Coloridos ===")

# Abre o arquivo HDF5 para frames coloridos e mascarados
with h5py.File(color_frames_h5_path, 'a') as color_h5, \
     h5py.File(masked_color_frames_h5_path, 'a') as masked_color_h5:
    
    # Pre-parse todas as anotações relevantes para os frames coloridos
    relevant_frames = set(range(total_frames))
    masks_per_frame = parse_annotations(annotations_path, relevant_frames, image_shape=(720, 1280, 3))  # Ajuste conforme necessário

    for i in range(num_chunks):
        start_frame = i * chunk_size
        end_frame = start_frame + chunk_size if i < num_chunks - 1 else total_frames

        # Extrair frames coloridos da parte atual
        print(f"\nProcessando parte {i+1} de {num_chunks}: frames {start_frame} a {end_frame}")
        color_frames_chunk, color_frame_names_chunk = extract_frames(color_video_path, start_frame, end_frame)

        # Verificar se frames foram extraídos
        if len(color_frames_chunk) == 0:
            print(f"Erro: Nenhum frame colorido extraído da parte {i+1}. Pulando...")
            continue

        # Salvar frames coloridos nos arquivos .h5
        dataset_color = f"color_chunk_{i+1}"
        save_frames_to_h5(color_h5, dataset_color, color_frames_chunk)
        dataset_color_names = f"color_chunk_{i+1}_names"
        save_frame_names(color_h5, dataset_color_names, color_frame_names_chunk)

        # Aplicar máscaras aos frames coloridos
        frame_ids = list(range(start_frame, end_frame))
        masked_color_frames_chunk, frames_with_masks_indices_chunk = apply_masks(color_frames_chunk, masks_per_frame, frame_ids)

        # Salvar frames com máscaras no arquivo .h5
        if masked_color_frames_chunk:
            dataset_masked = f"masked_color_chunk_{i+1}"
            save_frames_to_h5(masked_color_h5, dataset_masked, masked_color_frames_chunk)
            dataset_masked_names = f"masked_color_chunk_{i+1}_names"
            masked_frame_names_chunk = [f"frame_{idx:06d}" for idx in frames_with_masks_indices_chunk]
            save_frame_names(masked_color_h5, dataset_masked_names, masked_frame_names_chunk)
        
        # Limpar memória após salvar
        del color_frames_chunk
        del masked_color_frames_chunk
        del frames_with_masks_indices_chunk
        del color_frame_names_chunk
        print(f"Memória limpa após salvar a parte {i+1} dos frames coloridos")

# 3.5. Processamento dos Frames Térmicos

print("\n=== Processamento dos Frames Térmicos ===")

# Abre o arquivo HDF5 para frames térmicos
with h5py.File(thermal_frames_h5_path, 'a') as thermal_h5:
    for i in range(num_chunks):
        start_frame = i * chunk_size
        end_frame = start_frame + chunk_size if i < num_chunks - 1 else total_frames

        # Extrair frames térmicos da parte atual
        print(f"\nProcessando parte {i+1} de {num_chunks}: frames {start_frame} a {end_frame}")
        thermal_frames_chunk, thermal_frame_names_chunk = extract_frames(thermal_video_path, start_frame, end_frame)

        # Verificar se frames foram extraídos
        if len(thermal_frames_chunk) == 0:
            print(f"Erro: Nenhum frame térmico extraído da parte {i+1}. Pulando...")
            continue

        # Salvar frames térmicos nos arquivos .h5
        dataset_thermal = f"thermal_chunk_{i+1}"
        save_frames_to_h5(thermal_h5, dataset_thermal, thermal_frames_chunk)
        dataset_thermal_names = f"thermal_chunk_{i+1}_names"
        save_frame_names(thermal_h5, dataset_thermal_names, thermal_frame_names_chunk)

        # Limpar memória após salvar
        del thermal_frames_chunk
        del thermal_frame_names_chunk
        print(f"Memória limpa após salvar a parte {i+1} dos frames térmicos")

print("\nProcessamento completo.")



=== Processamento dos Frames Coloridos ===


Parseando anotações XML: 100%|██████████| 81992/81992 [00:00<00:00, 553647.70it/s]



Processando parte 1 de 5: frames 0 a 16398


Extraindo frames de video_colorido_concatenado_10fps.mp4 - Parte 0 a 16398: 100%|██████████| 16398/16398 [00:25<00:00, 646.26it/s]


Atualizando o dataset existente: color_chunk_1
